# Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
import pandas as pd

In [4]:
rate = pd.read_csv('ratings.csv')
mov =  pd.read_csv('movies.csv')
full = rate.merge(mov, on ='movieId', how = 'left')
full = full.drop(['genres','timestamp','userId'], axis='columns') 

In [5]:
full = full.groupby('title').mean()[['rating']].reset_index()

In [6]:
def movie_class(row):
    if row >= 4:
        return('Высокий')
    if row <= 4 and row > 2:
        return('Cредний')
    if row <= 2:
        return('Низкий')

In [7]:
full['class'] = full['rating'].apply(movie_class)
full.head()

,title,rating,class
0,'71 (2014),4.0,Высокий
1,'Hellboy': The Seeds of Creation (2004),4.0,Высокий
2,'Round Midnight (1986),3.5,Cредний
3,'Salem's Lot (2004),5.0,Высокий
4,'Til There Was You (1997),4.0,Высокий


# Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [184]:
words = pd.read_csv('keywords.csv')

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [228]:
def geolocator(row):
    edit = row.lower().split(' ')
    centr = ['москва','тула','ярославль']
    dv = ['петербург','псков','мурманск']
    sz = ['владивосток','сахалин', 'хабаровск']
    for i in edit:
        if i in centr:
            return('Центр')
        if i in dv:
            return('Дальний Восток')
        if i in sz:
            return('Северо-Запад')
    return('undefined')

In [231]:
words['region'] = words['keyword'].apply(geolocator)

In [230]:
print(words.loc[21445])
print(words.loc[21978])
print(words.loc[22920])


keyword         сахалин
shows             14494
region     Северо-Запад
Name: 21445, dtype: object
keyword    авито москва и московская область авто с пробе...
shows                                                  18149
region                                                 Центр
Name: 21978, dtype: object
keyword    интернет банк санкт петербург
shows                              18280
region                    Дальний Восток
Name: 22920, dtype: object


# Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [315]:
rate = pd.read_csv('ratings.csv')
mov =  pd.read_csv('movies.csv')
full = rate.merge(mov, on ='movieId', how = 'left')
full = full.drop(['genres','timestamp', 'userId'], axis='columns') 
full['rating'].astype('float')
full['Average Rating'] = full.groupby('movieId').rating.mean()                                                              
full = full.sort_values(by = 'Average Rating', ascending = False)

In [316]:
def year_sorting(row):
    to_split = row.lower().replace(')','').split('(')
    prod_year = to_split[-1]
    year_check = list(range(1950,2010))
    try:
        if int(prod_year) in year_check:
            return(prod_year)
        else:
            return('1900')
    except:
        return('1900')

In [317]:
full['Production Year'] = full['title'].apply(year_sorting)

In [323]:
full.loc[105,140]

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [140] of <class 'int'>